Maximum Likelihood Estimation
===
_by Ryan Delgado_

The purpose of this notebook is to introduce or refresh the concept of Maximum Likelihood Estimation (MLE). MLE is a commonly-used method of estimating parameters in a stastical model. It uses probability and numerical optimization to estimate parameters by answering the question "What is the most likely true value of the parameters that will result in the observations?"

This notebook uses the Normal Distribution as a basic application of MLE. The notebook assumes that you have at least a basic understanding of
+ Probability Theory, specifically joint probability
+ The Normal Distribution
+ Optimization

We'll briefly covers the intution behind MLE, and then apply the intuition to an example with the Normal Distribution.

Definition and Intuition of the Likelihood Function
===



MLE begins with the Likelihood Function, which is rooted in probability. Recall that the joint probability of events A and B both occurring is the product of their individual probabilities: $P(AB) = P(A)P(B)$. $P(AB)$ is the likelihood of both events A and B occurring, so $P(A)P(B)$ can be thought of as its _ikelihood fucntion_. We can generalize this by saying that the Likelihood Function of a set of a events is the joint probability of the events, or with the function:

$$ \prod_{i=1}^{n} P(x_{i}) $$ 

for events $ x_{1} $ through $ x_{n} $ where $ P(x_{i}) $ is the probability of event $ x_{i} $ occurring.






Application of MLE to the Normal Distribution
===

Say our model is the normal distribution, and we're fitting the model to a set of observations of a random variable $X$. Recall that the normal distribution is defined by the formula
$$ f\left(X, \mu, \sigma^{2}\right) = \frac{1}{\sqrt{2\sigma^{2}\pi}} e^-{\frac{\left(X-\mu\right)^2}{2\sigma^{2}}}$$ 

A set of $N$ observations of $X$ is a set of $N$ events, so the probability of observing all of these observations is the likelihood function:

$$ \prod_{i=1}^{N} P(X_{i}) = \prod_{i=1}^{N} \frac{1}{\sqrt{2\sigma^{2}\pi}} e^-{\frac{\left(X_{i}-\mu\right)^2}{2\sigma^{2}}}$$ 

This can be interpreted as the probability of this set of events occurring by sampling from a normally distributed population with unknown parameters $\mu$ and $\sigma$. 

The true values of $\mu$ and $\sigma$ are currently unknown and must be estimated from the data. This is where MLE becomes useful.

We'll use NumPy and SciPy's optimize toolkit:

In [1]:
import numpy as np
from scipy.optimize import minimize

Let's begin by defining the Normal Distribution in a Python function:

In [2]:
# Define the normal distribution
# parameters:
#   params - list: contains the mean and std deviation params for the normal distribution
#   x - 1d ndarray: contains the observations for the random variable X
def normal_dist(params, x):
    sig, mu = params
    return (1 / np.sqrt(2 * (sig ** 2) * np.pi)) * np.exp((-(x - mu) ** 2) / (2 * (sig ** 2)))

We'll define the likelihood function as the product the observation probabilities. I've also taken the natural log of the likelihood function so that it's easier to numerically optimize:

In [3]:
# Define the likelihood function
# parameters:
#   params - list: contains the mean and std deviation params for the normal distribution
#   x_array - 1d ndarray: an array of the sample
def norm_log_likelihood_function(params, x_array):
     return np.log(np.prod([normal_dist(params, x) for x in x_array]))

Optimizing the Log Likelihood function is equivalent to optimizing the Likelihood function because the Likelihood function monotonically increases. Let's define a lambda function that we'll optimize. I'll be minimizing the negative because it's numerically easier and standard practice:

In [4]:
nll = lambda *args: -norm_log_likelihood_function(*args)

Now we'll create a random, normally-distributed sample of 100 observations with a mean of approximately 5 and standard deviation of approximately 4:

In [5]:
sample = np.random.normal(5, 4, 100)
sample_std = np.std(sample)
sample_mean = np.mean(sample)


print("Sample standard deviation (found via np.std): %s" % str(sample_std))
print("Sample mean (found via np.mean): %s" % str(sample_mean))

Sample standard deviation (found via np.std): 4.15397283459
Sample mean (found via np.mean): 4.91738993881


We'll use <python>scipy.optimize.minimize </python>to estimate the parameters:

In [6]:
result = minimize(nll, [4,1], args=(sample))
mle_std = result["x"][0]
mle_mean = result["x"][1]

print("Sample standard deviation (found via MLE): %s" % str(mle_std))
print("Sample mean (found via MLE): %s" % str(mle_mean))

Sample standard deviation (found via MLE): 4.15397263173
Sample mean (found via MLE): 4.91738989301


Application of MLE to Ordinary Least Squares Regression
===

Ordinary Least Squares (OLS) is a method for estimating the parameters in a linear regression model. A linear regression model is a statistical model used to determine the linear relationship between dependent and independent variables. Example linear regression model:

$$ y_{i} = \beta_{0} + \beta_{1}x_{i}^{(1)} + \beta_{2}x_{i}^{(2)} + \varepsilon_{i} $$

Where $y_{i}$ is the dependent variable, $x_{i}^{(*)}$ are the independent variable, and $\varepsilon_{i}$ is the error term. We can more elegantly represent this using matrix notation:

$$ \large Y  \normalsize =  \large X\beta  \normalsize +  \large \varepsilon $$ 


$$ 
\begin{bmatrix} 
y_{1} \\ \vdots \\ y_{i} \\
\end{bmatrix}  =  
\begin{bmatrix} 
x_{1}^{(1)} & \dots  & x_{1}^{(n)} \\ 
\vdots & \ddots & \vdots 
\\ x_{i}^{(1)} & \dots  & x_{i}^{(n)} \\ 
\end{bmatrix} 
\begin{bmatrix} 
\beta_{1} \\ \vdots \\ \beta_{n} \\
\end{bmatrix} +
\begin{bmatrix} 
\varepsilon_{1} \\ \vdots \\ \varepsilon_{i} \\
\end{bmatrix}
$$

 





The $\beta$ parameters in this model can be estimated using MLE. If we assume that the error term is normally distributed with a mean of zero, our Likelihood function will be

$$ \prod_{i=1}^{N} \frac{1}{\sqrt{2\sigma^{2}\pi}} e^-{\frac{\left(y_{i}-X_{i}\beta\right)^2}{2\sigma^{2}}} $$

where $X_{i}$ is one observation from the independent variables matrix $X$, and $y_{i}$ is one observation of the dependent variable. Now the parameters to estimate include $\beta$, the vector of  parameters in the linear model, and $\sigma$, the standard error.

Let's slightly redefine our normal distribution to fit this:

In [20]:
# Define the function for normally distributed error terms in an OLS model
# parameters:
#   params - list: contains the 1d ndarray of beta params and std error parameter
#   x - 1d ndarray: contains the observation of the independent variables in matrix X
#   y - double: contains the observation for the dependent variable y
def normal_dist_ols(params, x, y):
    sig = params[0]
    beta = params[1:]
    return (1 / np.sqrt(2 * (sig ** 2) * np.pi)) * np.exp((-(y - np.dot(x,beta)) ** 2) / (2 * (sig ** 2)))

We'll generate a random sample of observations for $y$ and $X$. We'll only use one independent variable in $X$ for simplicity, in addition to the intercept column (which is a column of 1s only).

In [28]:
y_array = np.mat(np.random.normal(5,4,100)).T
x_matrix = np.mat(np.column_stack((np.ones(100), np.random.normal(5,4,100))))
print(y_array.shape)
print(x_matrix.shape)

(100, 1)
(100, 2)


The analytical derivation of the OLS estimator is:

$$ \large \beta =  \left(X^{T}X\right)^{-1}X^{T}y$$ 

In [40]:
beta_ols = np.linalg.inv(x_matrix.T * x_matrix) * (x_matrix.T * y_array)
print("Beta calculated with OLS: %s" % str(beta_ols))

Beta calculated with OLS: [[ 4.57113746]
 [ 0.17612323]]


Calculating the error vector: $ \varepsilon = y - X\beta $

In [41]:
error_ols = y_array - (x_matrix*beta_ols)
standard_error_ols = np.std(error_ols)
print("Standard error calculated with OLS: %s" % str(standard_error_ols))

Standard error calculated with OLS: 3.66545358683


In [24]:
# Define the likelihood function for OLS
# parameters:
#   params - list: contains the mean and std deviation params for the normal distribution
#   x_array - 1d ndarray: an array of the sample
def norm_log_likelihood_function_ols(params, x_matrix, y_array):
     return np.log(np.prod([normal_dist_ols(params, x, y) for x,y in zip(x_matrix, y_array)]))


In [38]:
nll = lambda *args: -norm_log_likelihood_function_ols(*args)

In [39]:
result = minimize(nll, [3,1,2], args=(x_matrix,y_array))
standard_error_mle = result.x[0]
beta_mle = result.x[1:]
print("Standard error calculated with MLE: %s" % str(standard_error_mle))
print("Beta calculated with MLE: %s" % str(beta_mle))


Standard error calculated with MLE: 3.66545355161
Beta calculated with MLE: [ 4.57113733  0.17612324]
